<h1>EECS 549 Final Project</h1>

Index set of Reddit posts 

Get preliminary set of relevant documents from the BM25 baseline and a subset of random documents for manual annotation 

In [1]:
import os 
from indexing import Indexer
from ranker import Ranker, BM25
from document_preprocessor import RegexTokenizer
import pickle
import pandas as pd
import json
import numpy as np 

<h2>Index document set</h2>

In [2]:
with open("../files/stopwords.txt") as f:
    words = f.read()
    stopwords = words.split()

In [3]:
document_preprocessor = RegexTokenizer("\\w+", lowercase = True)

In [ ]:
index = Indexer.create_index('BasicInvertedIndex', '../data/ask_reddit_posts.jsonl', document_preprocessor, stopwords, 20)

4615it [00:00, 239964.21it/s]
406it [00:00, 150192.93it/s]
4907it [00:00, 240162.54it/s]
892it [00:00, 157185.08it/s]
3591it [00:00, 216609.80it/s]
379it [00:00, 241220.21it/s]
4824it [00:00, 261449.59it/s]
4799it [00:00, 251508.35it/s]
2201it [00:00, 237666.07it/s]
820it [00:00, 245420.96it/s]
1910it [00:00, 250253.67it/s]
5094it [00:00, 256236.70it/s]
951it [00:00, 234523.94it/s]
278it [00:00, 226542.94it/s]
4672it [00:00, 259224.12it/s]
4776it [00:00, 255210.67it/s]
4711it [00:00, 256102.94it/s]
5028it [00:00, 270951.40it/s]
5002it [00:00, 272282.47it/s]
3664it [00:00, 268187.18it/s]
4552it [00:00, 239329.01it/s]
4617it [00:00, 276585.04it/s]
2043it [00:00, 174438.92it/s]
73it [00:00, 70533.10it/s]
4623it [00:00, 250811.89it/s]
4055it [00:00, 250462.44it/s]
4491it [00:00, 256822.13it/s]
206it [00:00, 216385.33it/s]
4644it [00:00, 275382.40it/s]
241it [00:00, 219458.81it/s]
4704it [00:00, 242651.65it/s]
451it [00:00, 248767.90it/s]
3400it [00:00, 247541.77it/s]
4899it [00:00, 276653.

In [ ]:
with open("../files/index_pickle", "wb") as f: 
    pickle.dump(index, f)

<h2>Get BM25 Rankings</h2>

In [ ]:
df = pd.read_csv("../data/ask_reddit.csv")

In [ ]:
ids_df = pd.read_csv("../data/id_map.csv", header = None)
ids_df.columns = ['new', 'old']

In [14]:
docids = list(ids_df['new'].values)

In [11]:
with open("../eval/evaluation_queries.csv") as f:
    lines = f.readlines()
    queries = [line.strip().replace('"', '') for line in lines[1:]]

In [12]:
bm25 = BM25(index)
ranker = Ranker(index, document_preprocessor, stopwords, bm25)

In [38]:
queries_to_rel = {}

for query in queries:
    docs = ranker.query(query)
    ids = [doc[0] for doc in docs][:20]
    random_ids = list(np.random.choice(docids, 10))
    ids.extend(random_ids)
    queries_to_rel[query] = ids

<h2>Format data</h2>

In [39]:
rel_df = pd.DataFrame.from_dict(queries_to_rel).T
rel_df.columns = [f"doc_{i}" for i in range(1, 31)]

In [ ]:
title_df = df[['title', 'id']]
for i in range(1, 31):
    target_col = rel_df[f'doc_{i}']
    rel_df[f'doc_{i}_old_id'] = target_col.apply(lambda s: ids_df[ids_df['new'] == s]['old'].values[0])
    rel_df[f'doc_{i}_title'] = rel_df[f'doc_{i}_old_id'].apply(lambda s: df[df['id'] == s]['title'].values[0])

In [ ]:
rel_df.to_csv("../eval/bm25_relevance_baseline.csv", header = None)